In [1]:
import random
import time

# Problem Initialization

In [2]:
max_city_number = 10
Gs = []
for NN in range(2,max_city_number+1):
    Gs.append([[random.randint(1,100) for e in range(NN+1)] for e in range(NN+1)])

# Effective Branch Calucation（Newton's Method）

In [3]:
def f(N,d,x):
    return (x**(d+1) - (N+1)*x + N)**2

def df(N,d,x):
    return 2*f(N,d,x)*((d+1)*x**d-(N+1))

def ddf(N,d,x):
    return 2*df(N,d,x)*((d+1)*x**d-(N+1))+2*f(N,d,x)*((d+1)*d*x**(d-1))

def solve(N,d):
    x = 1.9
    delta = 1.0
    count = 0
    while abs(delta)>0.000001 and count<100000:
        delta = df(N,d,x)/(ddf(N,d,x)+0.00001)
        x = x - delta
        count = count + 1
    return x

# Tree Structure

In [4]:
class node(object):
    def __init__(self,n,number=None,graph=None):
        self.pre = None
        self.city = number
        self.label = []
        self.child = []
        self.graph = graph
        self.n = n
        self.min_bound = [min(self.graph[i]) for i in range(self.n)]
        self.max_bound = [max([j for j in self.graph[i] if j!=float('inf')]) for i in range(self.n)]

    def add_child(self,number):
        tmp_node = node(n=self.n,number=number,graph=self.graph)
        tmp_node.pre = self
        tmp_node.label=[i for i in self.label]
        tmp_node.label.append(number)
        self.child.append(tmp_node)

    def get_routecost(self,label):
        g = 0
        if len(label)==(self.n):
            label.append(1)
        for i in range(0,len(label)-1):
            g = g+self.graph[label[i]-1][label[i+1]-1]
        return g

    def get_hcost(self,label):
        h = 0
        remain = city.difference(set(label))
        remain = list(remain)
        if len(remain)!=0:
            remain.append(label[-1])
        for i in remain:
            h = h+self.min_bound[i-1]
        return h

    def get_hmaxcost(self,label):
        h = 0
        remain = city.difference(set(label))
        remain = list(remain)
        if len(remain)!=0:
            remain.append(label[-1])
        for i in remain:
            h = h+self.max_bound[i-1]
        return h

  
    def get_estimatecost(self,label):
        remain = city.difference(set(label))
        remain = list(remain)
        remain.append(label[-1])
        if len(remain)==2:
            label.append(remain[0])
            label.append(1)
        f = self.get_routecost(label)
        f = f + self.get_hmaxcost(label)
        return f

    def get_fcost(self,label):
        remain = city.difference(set(label))
        remain = list(remain)
        remain.append(label[-1])
        if len(remain)==2:
            label.append(remain[0])
            label.append(1)
        f = self.get_routecost(label)
        f = f + self.get_hcost(label)
        return f

# A* Search Algorithm


In [5]:
resultsA = []
realans = []

def showOPEN(openlist):
    print("showOPEN")
    for i in openlist:
        print(i.label,i.get_routecost(i.label),i.get_fcost(i.label),i.get_estimatecost(i.label))
    print("XXXXXXXX")

for nidx in range(len(Gs)):
    print("----------i=%d------------"%nidx)
    graph = Gs[nidx]
    NN = len(graph)
    for idx in range(NN):
        graph[idx][idx] = float('inf')
    city = range(1,len(graph)+1)
    city = set(city)
    min_bound = [min(graph[i]) for i in range(len(graph))]

    tree = node(n=len(graph),number=1,graph=graph)
    tree.label.append(1)
    tree.cost=0

    start_time = time.time()
    open=[]
    open.append(tree)
    count = 0
    fcnt = 0
    ans = 0
    while len(open)>0:
        N = open[0]
        if len(N.label)==(len(city)-2):
            fcnt = fcnt+1
        del(open[0])
        count = count+1
        child_list = set(city).difference(set(N.label))
        if len(child_list)==0:
            ans = 1
            break
        for c in child_list:
            N.add_child(number=c)
        tmp = N.child
        for i in tmp:
            open.append(i)
        open = sorted(open,key= lambda x:x.get_fcost(x.label))
        #showOPEN(open)
    t = (time.time() - start_time)*1000
    print("--- %s msec ---" % (t))
    ans_route = N.get_routecost(N.label)

    if ans==1:
        print("RESULT:",N.label,ans_route,N.get_estimatecost(N.label))
        b = solve(count,NN-2)
        print("cities=%d, d=%d ,N= %d ,  b*=%f"%(len(graph)-1,NN-2,count,b))
        print("ROUTEs: %d"%(fcnt))
        resultsA.append((NN-2,count,b))
        realans.append(ans_route)
    else:
        print("FAILED")

----------i=0------------
--- 0.06031990051269531 msec ---
RESULT: [1, 3, 2, 1] 62 62
cities=2, d=1 ,N= 2 ,  b*=1.500000
ROUTEs: 1
----------i=1------------
--- 0.18095970153808594 msec ---
RESULT: [1, 4, 2, 3, 1] 133 133
cities=3, d=2 ,N= 3 ,  b*=1.154701
ROUTEs: 1
----------i=2------------
--- 0.4248619079589844 msec ---
RESULT: [1, 3, 4, 2, 5, 1] 83 83
cities=4, d=3 ,N= 4 ,  b*=1.077217
ROUTEs: 1
----------i=3------------
--- 1.6720294952392578 msec ---
RESULT: [1, 6, 4, 5, 3, 2, 1] 147 147
cities=5, d=4 ,N= 11 ,  b*=1.244666
ROUTEs: 4
----------i=4------------
--- 4.806995391845703 msec ---
RESULT: [1, 6, 7, 3, 2, 4, 5, 1] 182 182
cities=6, d=5 ,N= 22 ,  b*=1.308324
ROUTEs: 6
----------i=5------------
--- 22.773027420043945 msec ---
RESULT: [1, 7, 2, 3, 4, 6, 8, 5, 1] 110 110
cities=7, d=6 ,N= 42 ,  b*=1.353303
ROUTEs: 7
----------i=6------------
--- 1367.3608303070068 msec ---
RESULT: [1, 8, 3, 2, 9, 7, 5, 6, 4, 1] 205 205
cities=8, d=7 ,N= 419 ,  b*=1.760910
ROUTEs: 110
---------

In [6]:
resultsA

[(1, 2, 1.500000000000018),
 (2, 3, 1.1547005383839883),
 (3, 4, 1.0772173450173192),
 (4, 11, 1.244665954576957),
 (5, 22, 1.308324026895693),
 (6, 42, 1.3533030769655938),
 (7, 419, 1.760909527930808),
 (8, 875, 1.7722814815486114),
 (9, 1923, 1.7939096375992507)]

In [7]:
realans

[62, 133, 83, 147, 182, 110, 205, 134, 212]

# Gradient Ascent (Descent) Method

In [8]:
resultsGR = []
errors = []

for nidx in range(len(Gs)):
    print("----------i=%d------------"%nidx)
    graph = Gs[nidx]
    NN = len(graph)
    for idx in range(NN):
        graph[idx][idx] = float('inf')

    city = range(1,len(graph)+1)
    city = set(city)

    min_bound = [min(graph[i]) for i in range(len(graph))]

    tree = node(n=len(graph),number=1,graph=graph)
    tree.label.append(1)
    tree.cost=0

    start_time = time.time()

    count = 0
    N = tree
    n_city = len(city)
    while len(N.label) <= n_city+1:
        count = count+1
        if len(N.label)==(n_city+1):
            break
        child_list = set(city).difference(set(N.label))
        if len(child_list)==0:
            print("Fail")
            break
        for c in child_list:
            N.add_child(number=c)
        tmp = N.child

        open = []
        for i in tmp:
            open.append(i)
        open = sorted(open,key= lambda x:x.get_estimatecost(x.label))
        CN = open[0]

        if (CN.get_estimatecost(CN.label)<N.get_estimatecost(N.label)):
            N = CN
        else:
            print("CH fail break")
            break
    t = (time.time() - start_time)*1000
    answer=N.get_estimatecost(N.label)
    error = ((answer-realans[nidx])/realans[nidx])*100
    errors.append(error)
    print("RESULT:",N.label,answer,realans[nidx],"error:",error," %")
    print("cities=%d, d=%d ,N= %d ,  b*=%f"%(len(graph)-1,NN-2,count,solve(count,NN-2)))
    resultsGR.append((NN-2,count,solve(count,NN-2)))
    print("--- %s msec ---" % (t))
    print(sum(errors)/len(errors))

----------i=0------------
RESULT: [1, 3, 2, 1] 62 62 error: 0.0  %
cities=2, d=1 ,N= 2 ,  b*=1.500000
--- 0.28395652770996094 msec ---
0.0
----------i=1------------
RESULT: [1, 4, 2, 3, 1] 133 133 error: 0.0  %
cities=3, d=2 ,N= 3 ,  b*=1.154701
--- 0.20599365234375 msec ---
0.0
----------i=2------------
RESULT: [1, 3, 4, 2, 5, 1] 83 83 error: 0.0  %
cities=4, d=3 ,N= 4 ,  b*=1.077217
--- 0.4012584686279297 msec ---
0.0
----------i=3------------
RESULT: [1, 6, 3, 5, 2, 4, 1] 282 147 error: 91.83673469387756  %
cities=5, d=4 ,N= 5 ,  b*=1.046635
--- 0.7419586181640625 msec ---
22.95918367346939
----------i=4------------
RESULT: [1, 5, 2, 4, 3, 7, 6, 1] 267 182 error: 46.7032967032967  %
cities=6, d=5 ,N= 6 ,  b*=1.031310
--- 1.1742115020751953 msec ---
27.70800627943485
----------i=5------------
RESULT: [1, 7, 2, 3, 4, 6, 8, 5, 1] 110 110 error: 0.0  %
cities=7, d=6 ,N= 7 ,  b*=1.022505
--- 1.3310909271240234 msec ---
23.090005232862378
----------i=6------------
RESULT: [1, 4, 7, 5, 3, 

In [9]:
resultsGR

[(1, 2, 1.500000000000018),
 (2, 3, 1.1547005383839883),
 (3, 4, 1.0772173450173192),
 (4, 5, 1.046635139392451),
 (5, 6, 1.0313103067017242),
 (6, 7, 1.022504727328176),
 (7, 8, 1.0169685052002313),
 (8, 9, 1.0132571719231895),
 (9, 10, 1.0106462929716866)]

# Cost Search Algorithm


In [10]:
resultsC = []

for nidx in range(len(Gs)-3):
    print("----------i=%d------------"%nidx)
    graph = Gs[nidx]
    NN = len(graph)
    for idx in range(NN):
        graph[idx][idx] = float('inf')
    city = range(1,len(graph)+1)
    city = set(city)
    min_bound = [min(graph[i]) for i in range(len(graph))]

    tree = node(n=len(graph),number=1,graph=graph)
    tree.label.append(1)
    tree.cost=0

    start_time = time.time()
    open=[]
    open.append(tree)
  
    count = 0
    fcnt = 0
    ans = 0
    while len(open)>0:
        N = open[0]
        if len(N.label)==(len(city)-2):
            fcnt = fcnt+1
        del(open[0])
        count = count+1
        child_list = set(city).difference(set(N.label))
        if len(child_list)==0:
            ans = 1
            break
        for c in child_list:
            N.add_child(number=c)
        tmp = N.child
        for i in tmp:
            open.append(i)
        open = sorted(open,key= lambda x:x.get_routecost(x.label))
    #showOPEN(open)
    t = (time.time() - start_time)*1000
    print("--- %s msec ---" % (t))
    if ans==1:
        print("RESULT:",N.label,N.get_routecost(N.label),realans[nidx])
        b = solve(count,NN-2)
        print("cities=%d, d=%d ,N= %d ,  b*=%f"%(len(graph)-1,NN-2,count,b))
        print("ROUTEs: %d"%(fcnt))
        resultsC.append((NN-2,count,b))
    else:
        print("FAILED")

----------i=0------------
--- 0.16617774963378906 msec ---
RESULT: [1, 3, 2, 1] 62 62
cities=2, d=1 ,N= 4 ,  b*=2.500000
ROUTEs: 1
----------i=1------------
--- 0.5648136138916016 msec ---
RESULT: [1, 4, 2, 3, 1] 133 133
cities=3, d=2 ,N= 11 ,  b*=2.000000
ROUTEs: 3
----------i=2------------
--- 0.9660720825195312 msec ---
RESULT: [1, 3, 4, 2, 5, 1] 83 83
cities=4, d=3 ,N= 10 ,  b*=1.401020
ROUTEs: 4
----------i=3------------
--- 9.300947189331055 msec ---
RESULT: [1, 6, 4, 5, 3, 2, 1] 147 147
cities=5, d=4 ,N= 43 ,  b*=1.722347
ROUTEs: 14
----------i=4------------
--- 107.22494125366211 msec ---
RESULT: [1, 6, 7, 3, 2, 4, 5, 1] 182 182
cities=6, d=5 ,N= 215 ,  b*=2.047673
ROUTEs: 69
----------i=5------------
--- 172.0409393310547 msec ---
RESULT: [1, 7, 2, 3, 4, 6, 8, 5, 1] 110 110
cities=7, d=6 ,N= 242 ,  b*=1.806140
ROUTEs: 49


In [11]:
resultsC

[(1, 4, 2.4999999999995124),
 (2, 11, 2.0),
 (3, 10, 1.4010196653276958),
 (4, 43, 1.722347059926734),
 (5, 215, 2.0476725110792193),
 (6, 242, 1.8061398392728834)]

# Depth First Algorithm


In [12]:
resultsV = []

for nidx in range(len(Gs)-3):
    print("----------i=%d------------"%nidx)
    graph = Gs[nidx]
    NN = len(graph)
    for idx in range(NN):
        graph[idx][idx] = float('inf')
    city = range(1,len(graph)+1)
    city = set(city)

    tree = node(n=len(graph),number=1,graph=graph)
    tree.label.append(1)
    tree.cost=0

    start_time = time.time()
    open=[]
    paths = []
    count = 0
    fcnt = 0
    open.append(tree)
    while len(open)>0:
        if len(open)==0:
            break
        N = open.pop()

        if len(N.label)==(len(graph)):
            N.label.append(1)
            paths.append(N)
            fcnt = fcnt+1
        paths=sorted(paths,key= lambda x:x.get_routecost(x.label))

        count = count+1
        child_list = set(city).difference(set(N.label))
        if len(child_list)==0:
            continue
        for c in child_list:
            N.add_child(number=c)
        tmp = N.child

        for i in tmp:
            open.append(i)
    t = (time.time() - start_time)*1000
    ans_route = paths[0].get_routecost(paths[0].label)
    print("RESULT:",paths[0].label,ans_route,realans[nidx])
    print("cities=%d, d=%d ,N= %d ,  b*=%f"%(len(graph)-1,NN-2,count,solve(count,NN-2)))
    resultsV.append((NN-2,count,solve(count,NN-1)))
    print("ROUTEs:%d"%(fcnt))
    print("--- %s msec ---" % (t))

----------i=0------------
RESULT: [1, 3, 2, 1] 62 62
cities=2, d=1 ,N= 5 ,  b*=3.000000
ROUTEs:2
--- 0.2689361572265625 msec ---
----------i=1------------
RESULT: [1, 4, 2, 3, 1] 133 133
cities=3, d=2 ,N= 16 ,  b*=2.380476
ROUTEs:6
--- 0.30493736267089844 msec ---
----------i=2------------
RESULT: [1, 3, 4, 2, 5, 1] 83 83
cities=4, d=3 ,N= 65 ,  b*=2.545822
ROUTEs:24
--- 2.930164337158203 msec ---
----------i=3------------
RESULT: [1, 6, 4, 5, 3, 2, 1] 147 147
cities=5, d=4 ,N= 326 ,  b*=2.843770
ROUTEs:120
--- 62.73174285888672 msec ---
----------i=4------------
RESULT: [1, 6, 7, 3, 2, 4, 5, 1] 182 182
cities=6, d=5 ,N= 1957 ,  b*=3.182235
ROUTEs:720
--- 1794.2638397216797 msec ---
----------i=5------------
RESULT: [1, 7, 2, 3, 4, 6, 8, 5, 1] 110 110
cities=7, d=6 ,N= 13700 ,  b*=3.536788
ROUTEs:5040
--- 94124.37701225281 msec ---


In [13]:
resultsV

[(1, 5, 1.4142135623731),
 (2, 16, 1.6198059006387417),
 (3, 65, 1.906090350669923),
 (4, 326, 2.224740190162272),
 (5, 1957, 2.557324118662959),
 (6, 13700, 2.897021739443863)]